In [ ]:
!wget http://bit.ly/3ZLyF82 -O CSS.css -q

from IPython.core.display import HTML

with open("./CSS.css", "r") as file:
    custom_css = file.read()

HTML(custom_css)

In [ ]:
print("Total files and folders in the dataset...\n")
!ls ../input/facial-keypoints-detection -GFlash --color

# <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#4285F4;overflow:hidden">Facial Keypoints Detection</div>

<div align='center'>
<img src="https://miro.medium.com/max/1200/1*a6kXOpZQ4abIk0EfIkKOpw.jpeg" width="700" height="700" style="max-width: 546px; padding-top: 10px;padding-bottom: 10px">
</div>

<div style="font-size:18px; font-family:Georgia;">
<ul style=“list-style-type:circle;”>
    <li>The marking of important areas of the face, such as the eyes, corners of the mouth, and nose, that are relevant for a variety of tasks, such as face filters, emotion recognition, and pose recognition, using convolutional neural network and computer vision techniques to perform facial Keypoint detection.</li>
    <li> It entails forecasting the Facial Key points coordinates for a particular face, such as nose tip, the center of eyes, and so on. To recognize facial key points, we use a Convolutional Neural Network-based model using Autoencoders. </li>
    <li>Convolutional Neural Networks (CNN) has a deep structure that allows them to extract high-level information and provide better precision when identifying each important point. Convolutional networks are designed to anticipate all points at the same time.</li>
</ul>
</div>

<div style="font-size:18px; font-family:Georgia;">
I have made a generous effort to build the model and tried to explain each and every step in great depth.<br>

I have built two models one a Simple Neural Network and other a Convolutional Neural Network and compared thr result of the to models.
</div>

# <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#4285F4;overflow:hidden">Libraries</div>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt


import os
import zipfile
import cv2

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#4285F4;overflow:hidden">Import and visualize train and test data</div>

In [ ]:
train = pd.read_csv('/kaggle/input/facial-keypoints-detection/training.zip', compression='zip')
test = pd.read_csv('/kaggle/input/facial-keypoints-detection/test.zip', compression='zip')
IdLookupTable = pd.read_csv('../input/facial-keypoints-detection/IdLookupTable.csv',header=0, sep=',', quotechar='"')

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
m, n = train.shape
m1, n1 = test.shape
print(f'Rows in training data: {m}\nColumns in train data: {n}')
print(f'Rows in test data: {m1}\nColumns in test data: {n1}')

# <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#4285F4;overflow:hidden"> Check for missing values</div>

In [ ]:
train.isnull().sum()

<div style="font-size:18px; font-family:Georgia;">
As there are large mising values so deleting them will evidently reduce our data and hamper our predictions.<br>
So I will replace the values with their previous values in that row.<br>
As replacing with mean won't guarrenty that the newly filled positions will be accurately aligned the attributes.
</div>


## <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#4285F4;overflow:hidden">Fill the missing values using ffill</div>

In [ ]:
train.fillna(method = 'ffill', inplace=True) # forward fill

<div style="font-size:18px; font-family:Georgia;">Sanity check (again check for missing values)</div>


In [ ]:
train.isnull().sum().any()

## <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#4285F4;overflow:hidden">Now analyze the Image column.</div>

In [ ]:
img_height, img_width = 96, 96

In [ ]:
type(train['Image'][0])

<div style="font-size:18px; font-family:Georgia;">As the type of the entried in the Image column is string so we will need to convert it into string.</div>

In [ ]:
train['Image'][100].split(' ')

In [ ]:
img = []
# Iterate over all the rows and split the individual entries of Image column
# if there are any blank spaces i.e. null values in the pixels then simply replace it with 0
for i in range(m):
    face_pixel = np.array(train['Image'][i].split(' '), dtype='float')
    face_pixel = np.reshape(face_pixel, (img_height,img_width,1))
    face_pixel /=255
    img.append(face_pixel)
img = np.array(img)

In [ ]:
print(img[0])

<div style="font-size:18px; font-family:Georgia;">Now all the entries in Image are converted into a list, now we should convert all the entries in into numpy array and convert the dtype into float so that we will be able to perform calculations over them.</div>

# <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#4285F4;overflow:hidden">Separate the X_train and y_train data</div>

In [ ]:
X_train = img
train.drop('Image', axis = 1, inplace=True)
y_train = []

for i in range(len(train)):
    y = train.iloc[i,:].values
    y_train.append(y)

In [ ]:
y_train = np.array(y_train,dtype = 'float')

In [ ]:
X_train.shape, y_train.shape

In [ ]:
y_train

<div style="font-size:18px; font-family:Georgia;">Let's see the test images</div>

In [ ]:
plt.imshow(X_train[1550])
plt.show()

<div style="font-size:18px; font-family:Georgia;">Let's see the images with facial key points</div>

In [ ]:
def image_with_keypoints(df, index):
    image = plt.imshow(X_train[index],cmap='gray')
    l = []
    for i in range(1,31,2):
        l.append(plt.plot(y_train[index][i-1], y_train[index][i], 'r*'))
        
    return image, l

In [ ]:
# plt.plot(y_train.loc[0][0], y_train.loc[0][0], 'bo')

In [ ]:
fig = plt.figure(figsize=(8, 8))
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

for i in range(16):
    ax = fig.add_subplot(4, 4, i + 1, xticks=[], yticks=[])
    image_with_keypoints(X_train, i)

plt.show()

<div style="font-size:18px; font-family:Georgia;">
All the preparations are done<br>
Start building our model
</div>



# <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#4285F4;overflow:hidden">Start building the model</div>

In [ ]:
import tensorflow as tf
from tensorflow.keras import callbacks
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPool2D, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import RootMeanSquaredError
print("Successfully Imported")

<div style="font-size:18px; font-family:Georgia;">
This will be a baseline model. This model will be built and later on we will fine tune the model.<br>
Also I am gonna build a CNN to compare the results.
</div>

## <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#4285F4;overflow:hidden">Neural Network with dropout</div>

In [ ]:
model = Sequential()
model.add(Flatten(input_shape=(96, 96, 1)))
model.add(Dense(128, activation='relu'))
Dropout(0.5)
model.add(Dense(128, activation='relu'))
Dropout(0.5)
model.add(Dense(64, activation='relu'))
model.add(Dense(30))

model.summary()

In [ ]:
model.compile(optimizer='Adam',  metrics=[RootMeanSquaredError(), 'accuracy'], loss='mae')

<div style="font-size:18px; font-family:Georgia;">Now our model is defined and we will train it by calling fit method. I ran it for 500 iteration keeping batch size and validtion set size as 20% ( 20% of the training data will be kept for validating the model ).</div>

In [ ]:
%%time
history = model.fit(X_train,y_train,epochs = 200)

## <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#4285F4;overflow:hidden">Convolutional Neural Network</div>

In [ ]:
model = Sequential()

model.add(Conv2D(128, kernel_size=(3, 3), input_shape=(img_height, img_width, 1),
                activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=(3, 3),activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3, 3),activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(30))
model.summary()

In [ ]:
model.compile(optimizer='Adam',  metrics=[RootMeanSquaredError(), 'accuracy'], loss='mae')

In [ ]:
%%time
history = model.fit(X_train,y_train,epochs = 50, batch_size = 32,validation_split = 0.2)

In [ ]:
print(history.history.keys())

# <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#4285F4;overflow:hidden">Plot the losses and accuracy</div>

In [ ]:
def plot_loss(hist,name,plt):
    '''
    RMSE_TF: if True, then RMSE is plotted with original scale 
    '''
    loss = hist['loss']
    val_loss = hist['val_loss']
    plt.plot(loss,"--",linewidth=3,label="train:"+name)
    plt.plot(val_loss,linewidth=3,label="val:"+name)

plot_loss(history.history,"CNN",plt)
plt.legend()
plt.grid()
plt.yscale("log")
plt.xlabel("epoch")
plt.ylabel("log loss")
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(acc,"--",linewidth=3,label="train : CNN")
plt.plot(val_acc,linewidth=3,label="val : CNN")
plt.legend()
plt.grid()
plt.yscale("log")
plt.xlabel("epoch")
plt.ylabel("accuracy")
plt.show()

# <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#4285F4;overflow:hidden">Prepare test data</div>

In [ ]:
test_images = []
for i in range(len(test)):
    item = np.array(test['Image'][i].split(' '), dtype='float')
    item = np.reshape(item, (img_height, img_width, 1))
    item /= 255
    test_images.append(item)
    
test_images = np.array(test_images, dtype='float')

In [ ]:
plt.imshow(test_images[0].reshape(96, 96))
plt.show()

# <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#4285F4;overflow:hidden">Predict on test data</div>

In [ ]:
predict = model.predict(test_images)

# <div style="padding:20px;color:white;margin:0;font-size:30px;font-family:Georgia;text-align:center;display:fill;border-radius:5px;background-color:#4285F4;overflow:hidden">Submission</div>

In [ ]:
feature_names = list(IdLookupTable['FeatureName'])
image_ids = list(IdLookupTable['ImageId']-1)
row_ids = list(IdLookupTable['RowId'])

feature_list = []
for feature in feature_names:
    feature_list.append(feature_names.index(feature))

In [ ]:
predictions = []
for x,y in zip(image_ids, feature_list):
    predictions.append(predict[x][y])

In [ ]:
row_ids = pd.Series(row_ids, name = 'RowId')
locations = pd.Series(predictions, name = 'Location')
locations = locations.clip(0.0,96.0)
submission_result = pd.concat([row_ids,locations],axis = 1)
submission_result.to_csv('submission.csv',index = False)

In [ ]:
# # pd.read_csv('/kaggle/input/facial-keypoints-detection/SampleSubmission.csv')
# pd.read_csv('/kaggle/input/facial-keypoints-detection/IdLookupTable.csv')
# FeatureName = list(IdLookupTable['FeatureName'])
# ImageID = list(IdLookupTable['ImageId']-1)
# RowId = list(IdLookupTable['RowId'])

# # prediction result
# predict = list(predict)


# Data = []
# for i in list(FeatureName):
#     Data.append(FeatureName.index(i))
    

# Data_Pre = []
# for x,y in zip(ImageID,Data):
#     Data_Pre.append(predict[x][y])


# RowId = pd.Series(RowId,name = 'RowId')
# Location = pd.Series(Data_Pre,name = 'Location')

# submission = pd.concat([RowId,Location],axis = 1)
# submission.to_csv('./submission.csv',index=False)